# Projekt z BPC-UIN č.1
## Autor: Jan Tomšej, 256421
+ Import knihoven pro práci s obrázky

In [14]:
from IPython.display import Image #library for image manipulation
import time, codecs, math

+ Deklarace třídy CNode pro políčko bludiště ("jeden pixel"). *Pro všechny algoritmy (informované i neinformované) je tato třída společná*.

In [23]:
from PIL import Image #library for image manipulation
import time, codecs, math

class T_Place():
  # Labyrinth place
  def __init__(self):
    self.Wall=False          # Place is wall - labyrinth image black point
    self.Visited=False       # Place was already visited
    self.Path=False          # Is part of final path
    self.Distance=0          # Distance from entrance place
    self.Heuristics=0        # h(x) - Heuristics variable
    self.Pred=None           # T_RowCol - Row and Col of predecessor place

class T_RowCol():
  # Information about Row and Col
  def __init__(self):
    self.Row=0
    self.Col=0
      
Lab_Width=0          # Labyrinth width - is read from labyrinth image
Lab_Height=0         # Labyrinth height - is read from labyrinth image
Lab=None             # Labyrinth - 2-dimensional list of T_Place
Lab_Entrance=None    # T_RowCol of entrance place - labyrinth image blue point
Lab_Exit=None        # T_RowCol of exit place - labyrinth image red point

FIFO=None            # FIFO (First In, First Out) queue list - for BFS algorithm only
FIFO_Pointer=-1      # The newest element in FIFO queue list

class T_Algorithm_Info():
  # Information for used algorithm
  def __init__(self):
    self.Visited=0   # Number of visited places
    self.Distance=0  # Length of final path - distance of exit place from entrance place
    self.Duration=0  # Duration [s]

DFS_Info=T_Algorithm_Info()
BFS_Info=T_Algorithm_Info()
Manhattan_Info=T_Algorithm_Info()

def File_Open(Name,Mode):
  # Open text file
  try:
    f=codecs.open(Name,Mode,"utf-8")
    return f
  except Exception as e:
    return None

def File_Close(f):
  # Close text file
  try:
    if f is not None: 
      if not f.closed: 
        f.close
  except Exception as e:
    pass

def Labyrinth_Read():
  # Reading the labyrinth from file 'Labyrinth.bmp'
  global Lab, Lab_Height, Lab_Width, Lab_Entrance, Lab_Exit
  global FIFO, FIFO_Pointer
  im = Image.open("Labyrinth.bmp")
  Lab_Width=im.size[0]
  Lab_Height=im.size[1]
  # Create empty Labyrinth
  if Lab is None:
    Lab=[]
    for Row in range(Lab_Height):
      C=[]
      for Col in range(Lab_Width):
        Place=T_Place()
        Place.Pred=T_RowCol()
        C.append(Place)
      Lab.append(C)
  # Read places information from image
  for Row in range(Lab_Height):
    for Col in range(Lab_Width):
      pixel=im.getpixel((Col,Row))
      Lab[Row][Col].Wall=(pixel==(0,0,0))   # Black pixel = wall
      Lab[Row][Col].Visited=False
      Lab[Row][Col].Path=False
      Lab[Row][Col].Distance=99999
      if pixel==(0,0,255):
        # Blue pixel = entrance
        Lab_Entrance=T_RowCol()
        Lab_Entrance.Row=Row
        Lab_Entrance.Col=Col
      if pixel==(255,0,0):
        # Red pixel = exit
        Lab_Exit=T_RowCol()
        Lab_Exit.Row=Row
        Lab_Exit.Col=Col
  # Initialize/Clear FIFO for BFS algorithm
  if FIFO is None:
    FIFO=[]
    for i in range(Lab_Height*Lab_Width):
      Place_RowCol=T_RowCol()
      FIFO.append(Place_RowCol)
  FIFO_Pointer=-1
  # Set heuristics value for Manhattan algorithm
  for Row in range(Lab_Height):
    for Col in range(Lab_Width):
      # K * (distance from entrance + distance from exit)
      Lab[Row][Col].Heuristics=round(20*( math.sqrt((Row - Lab_Entrance.Row) ** 2 + (Col - Lab_Entrance.Col) ** 2) + math.sqrt((Row - Lab_Exit.Row) ** 2 + (Col - Lab_Exit.Col) ** 2) ))

def Labyrinth_Path_Construction():
  # Move from exit to entrance using .Pred information to set place .Path property
  global Lab, Lab_Height, Lab_Width, Lab_Entrance, Lab_Exit
  Row=Lab_Exit.Row
  Col=Lab_Exit.Col
  while Row!=Lab_Entrance.Row or Col!=Lab_Entrance.Col:
    Lab[Row][Col].Path = True
    Row_Tmp = Lab[Row][Col].Pred.Row
    Col_Tmp = Lab[Row][Col].Pred.Col
    Row = Row_Tmp
    Col = Col_Tmp

def Labyrinth_Result(Algorithm, Alg_Info):
  # Create bitmap image, count number of visited places
  img = Image.new( 'RGB', (Lab_Width,Lab_Height), "black") # Create a new black image
  pixels = img.load() # Create the pixel map
  Alg_Info.Visited=0
  for Row in range(Lab_Height):
    for Col in range(Lab_Width):
      if Lab[Row][Col].Wall:
        pixels[Col,Row] = (0, 0, 0)
      elif Lab[Row][Col].Path:
        Alg_Info.Visited=Alg_Info.Visited+1
        pixels[Col,Row] = (0, 128, 0)
      elif Lab[Row][Col].Visited:
        Alg_Info.Visited=Alg_Info.Visited+1
        pixels[Col,Row] = (192, 192, 255)
      else:
        pixels[Col,Row] = (255, 255, 255)
  img.save('Labyrinth_'+Algorithm+'.bmp')
#  img.show()
#  input("Press Enter to continue...")

# ***********************************************
# DFS algorithm
# ***********************************************
def DFS():
  global Lab_Entrance
  Lab[Lab_Entrance.Row][Lab_Entrance.Col].Distance=0
  DFS_Find(Lab_Entrance.Row, Lab_Entrance.Col)

def DFS_Find(Row, Col):
  # Returns 'True' only if exit place was reached
  global Lab, Lab_Height, Lab_Width, Lab_Exit
  Lab[Row][Col].Visited = True
  Lab[Row][Col].Path = True
  if Row==Lab_Exit.Row and Col==Lab_Exit.Col:
    # Exit was reached
    return True
  if Row<Lab_Height-1:
    # Try down
    P=Lab[Row+1][Col]
    if not P.Wall and not P.Visited:
      P.Distance=Lab[Row][Col].Distance+1
      if DFS_Find(Row+1, Col):
        return True
      # - returned from path down
  if Col>0:
    # Try left
    P=Lab[Row][Col-1]
    if not P.Wall and not P.Visited:
      P.Distance=Lab[Row][Col].Distance+1
      if DFS_Find(Row, Col-1):
        return True
      # - returned from path left
  if Col<Lab_Width-1:
    # Try right
    P=Lab[Row][Col+1]
    if not P.Wall and not P.Visited:
      P.Distance=Lab[Row][Col].Distance+1
      if DFS_Find(Row, Col+1):
        return True
      # - returned from path right
  if Row>0:
    # Try up
    P=Lab[Row-1][Col]
    if not P.Wall and not P.Visited:
      P.Distance=Lab[Row][Col].Distance+1
      if DFS_Find(Row-1,Col):
        return True
      # - returned from path up
  Lab[Row][Col].Path=False
  return False

# ***********************************************
# BFS algorithm
# ***********************************************
def FIFO_Empty():
  # Is FIFO empty?
  global FIFO, FIFO_Pointer
  return (FIFO_Pointer==-1)

def FIFO_Push(Row, Col):
  # Push Row and Col information to FIFO
  global FIFO, FIFO_Pointer
  FIFO_Pointer=FIFO_Pointer+1
  FIFO[FIFO_Pointer].Row=Row
  FIFO[FIFO_Pointer].Col=Col

def FIFO_Pop():
  # Get Row and Col information from FIFO
  global FIFO, FIFO_Pointer
  RowCol=[]
  RowCol.append(FIFO[0].Row)
  RowCol.append(FIFO[0].Col)
  for i in range(FIFO_Pointer):
    FIFO[i].Row=FIFO[i+1].Row
    FIFO[i].Col=FIFO[i+1].Col
  FIFO_Pointer=FIFO_Pointer-1  
  return RowCol

def BFS():
  global Lab, Lab_Height, Lab_Width, Lab_Entrance, Lab_Exit
  global FIFO, FIFO_Pointer
  # Start from entrance place
  Row=Lab_Entrance.Row
  Col=Lab_Entrance.Col
  Lab[Row][Col].Path=True
  Lab[Row][Col].Visited=True
  Lab[Row][Col].Distance=0
  FIFO_Pointer=-1
  FIFO_Push(Row, Col)
  while not FIFO_Empty():
    # Get oldest place
    RowCol=FIFO_Pop()
    Row=RowCol[0]
    Col=RowCol[1]
    # Push all its unvisited neighbours to FIFO
    if Row<Lab_Height-1:
      P=Lab[Row+1][Col]
      if not P.Wall and not P.Visited:
        # Down
        P.Visited=True
        P.Distance=Lab[Row][Col].Distance+1
        P.Pred.Row=Row
        P.Pred.Col=Col
        FIFO_Push(Row+1, Col)
    if Col>0:
      P=Lab[Row][Col-1]
      if not P.Wall and not P.Visited:
        # Left
        P.Visited=True
        P.Distance=Lab[Row][Col].Distance+1
        P.Pred.Row=Row
        P.Pred.Col=Col
        FIFO_Push(Row, Col-1)
    if Col<Lab_Width-1:
      P=Lab[Row][Col+1]
      if not P.Wall and not P.Visited:
        # Right
        P.Visited=True
        P.Distance=Lab[Row][Col].Distance+1
        P.Pred.Row=Row
        P.Pred.Col=Col
        FIFO_Push(Row, Col+1)
    if Row>0:
      P=Lab[Row-1][Col]
      if not P.Wall and not P.Visited:
        # Up
        P.Visited=True
        P.Distance=Lab[Row][Col].Distance+1
        P.Pred.Row=Row
        P.Pred.Col=Col
        FIFO_Push(Row-1, Col)
  Labyrinth_Path_Construction()

# ***********************************************
# A* (Manhattan) algorithm
# ***********************************************
def Manhattan():
  global Lab, Lab_Height, Lab_Width, Lab_Entrance, Lab_Exit
  Row=Lab_Entrance.Row
  Col=Lab_Entrance.Col
  Lab[Row][Col].Path=True
  Lab[Row][Col].Distance=0        # ALL OTHER PLACES have Distance=99999 (infinity) !
  Finished=False
  while not Finished:
    # Find place with minimal distance+heuristics from entrance place
    Dist_Min = 99999
    for Row in range(Lab_Height):
      for Col in range(Lab_Width):
        if Lab[Row][Col].Distance!=99999:
          if not Lab[Row][Col].Wall and not Lab[Row][Col].Visited and Lab[Row][Col].Distance+Lab[Row][Col].Heuristics<Dist_Min:
            Dist_Min=Lab[Row][Col].Distance+Lab[Row][Col].Heuristics
            Row_Min=Row
            Col_Min=Col
    Row=Row_Min
    Col=Col_Min
    if Row<Lab_Height-1:
      # Down
      P=Lab[Row+1][Col]
      if not P.Wall and not P.Visited:
        # Shorter distance from entrance place?
        if Lab[Row][Col].Distance+1<P.Distance:
          P.Distance=Lab[Row][Col].Distance+1
          P.Pred.Row=Row
          P.Pred.Col=Col
    if Col>0:
      # Left
      P=Lab[Row][Col-1]
      if not P.Wall and not P.Visited:
        if Lab[Row][Col].Distance+1<P.Distance:
          P.Distance=Lab[Row][Col].Distance+1
          P.Pred.Row=Row
          P.Pred.Col=Col
    if Col<Lab_Width-1:
      # Right
      P=Lab[Row][Col+1]
      if not P.Wall and not P.Visited:
        if Lab[Row][Col].Distance+1<P.Distance:
          P.Distance=Lab[Row][Col].Distance+1
          P.Pred.Row=Row
          P.Pred.Col=Col
    if Row>0:
      # Up
      P=Lab[Row-1][Col]
      if not P.Wall and not P.Visited:
        if Lab[Row][Col].Distance+1<P.Distance:
          P.Distance=Lab[Row][Col].Distance+1
          P.Pred.Row=Row
          P.Pred.Col=Col
    Lab[Row][Col].Visited=True
    # Finished?
    if Row==Lab_Exit.Row and Col==Lab_Exit.Col:
      # Yes - is at exit point
      Finished=True
    else:
      # does non-visited place exist?
      Finished=True
      for Row in range(Lab_Height):
        for Col in range(Lab_Width):
          if not Lab[Row][Col].Wall and not Lab[Row][Col].Visited:
            Finished=False
            break
        if not Finished:
          break
  Labyrinth_Path_Construction()

Labyrinth_Read()
Start_Time=time.time()
DFS()
DFS_Info.Duration=time.time()-Start_Time
DFS_Info.Distance=Lab[Lab_Exit.Row][Lab_Exit.Col].Distance+1    # +1, because Entrance place has Distance=0
Labyrinth_Result('DFS',DFS_Info)

Labyrinth_Read()
Start_Time=time.time()
BFS()
BFS_Info.Duration=time.time()-Start_Time
BFS_Info.Distance=Lab[Lab_Exit.Row][Lab_Exit.Col].Distance+1    # +1, because Entrance place has Distance=0
Labyrinth_Result('BFS',BFS_Info)

Labyrinth_Read()
Start_Time=time.time()
Manhattan()
Manhattan_Info.Duration=time.time()-Start_Time
Manhattan_Info.Distance=Lab[Lab_Exit.Row][Lab_Exit.Col].Distance+1   # +1, because Entrance place has Distance=0
Labyrinth_Result('Manhattan',Manhattan_Info)

# File="C:\Users\tomse\3.rocnik\BPC-UIN\Result.txt"
# f=File_Open(File,'w')
print('|-------------------------------------------------------------------|'+'\r\n')
print('| Algoritmus     | Delka cesty | Prozkoumanych uzlu | Doba behu (s) |'+'\r\n')
print('|----------------|-------------|--------------------|---------------|'+'\r\n')
S='| DFS            |'+str(DFS_Info.Distance).rjust(4)+'         |'+str(DFS_Info.Visited).rjust(4)+'                | '+str(round(DFS_Info.Duration,6)).ljust(8)+'      |'
print(S+'\r\n')
S='| BFS            |'+str(BFS_Info.Distance).rjust(4)+'         |'+str(BFS_Info.Visited).rjust(4)+'                | '+str(round(BFS_Info.Duration,6)).ljust(8)+'      |'
print(S+'\r\n')
S='| A* (Manhattan) |'+str(Manhattan_Info.Distance).rjust(4)+'         |'+str(Manhattan_Info.Visited).rjust(4)+'                | '+str(round(Manhattan_Info.Duration,6)).ljust(8)+'      |'
print(S+'\r\n')
print('|-------------------------------------------------------------------|'+'\r\n')
# File_Close(f)

|-------------------------------------------------------------------|

| Algoritmus     | Delka cesty | Prozkoumanych uzlu | Doba behu (s) |

|----------------|-------------|--------------------|---------------|

| DFS            | 153         | 336                | 0.000165      |

| BFS            |  99         | 421                | 0.000371      |

| A* (Manhattan) | 103         | 315                | 0.009335      |

|-------------------------------------------------------------------|

